In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

In [3]:
train_data_dir = '/content/drive/MyDrive/tomato/train'
validation_data_dir = '/content/drive/MyDrive/tomato/val'


In [4]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 10

processing the data using imagedatagenerator

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,  # it's a set
    class_mode='categorical') # lable are not hard coded

Found 10000 images belonging to 10 classes.


Build the ResNet50 model

In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

94765736/94765736 [==============================] - 0s 0us/step


compile model

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    classes = None)

Found 10000 images belonging to 10 classes.


In [10]:
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    classes =train_generator.class_indices)

Found 1000 images belonging to 10 classes.


build the resnet50 model

In [11]:
model.save('/content/drive/MyDrive/your_model_folder/resnet_tomato_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
from tensorflow.keras.models import load_model
model_path = '/content/drive/MyDrive/your_model_folder/resnet_tomato_model.h5'
loaded_model = load_model(model_path)

In [13]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 49280650 (187.99 MB)
Trainable params: 49227530 (187.79 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [ ]:
validation_loss, validation_accuracy = loaded_model.evaluate(validation_generator)

print(f'Validation Accuracy: {validation_accuracy * 100:.2f}%')
print(f'Validation Loss: {validation_loss}')

 3/32 [=>............................] - ETA: 2:46 - loss: 2.5879 - accuracy: 0.0938